In [1]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import network


In [7]:
def calculateMatrices(components, nodeCount):

    # use global scope variables for component counts
    global voltageCount, inductorCount

    # calculate g2 components
    g2Count = voltageCount + inductorCount
    print("Group 2 count:", g2Count)

    # calculate matrix size
    matrixSize = nodeCount + g2Count - 1
    print("Matrix size:", matrixSize, "\n")

    # define Matrices
    A = np.zeros((matrixSize, matrixSize))
    b = np.zeros(matrixSize)

    # Group 2 component index
    g2Index = matrixSize - g2Count

    # loop through all components
    for component in components:
        # store component info in temporary variable
        high = component.high
        low = component.low
        value = component.value

        if component.comp_type == 'R':
            # affects G-matrix of A
            # diagonal self-conductance of node
            if high != 0:
                A[high - 1][high - 1] = A[high - 1][high - 1] + 1/value
            if low != 0:
                A[low - 1][low - 1] = A[low - 1][low - 1] + 1/value

            # mutual conductance between nodes
            if high != 0 and low != 0:
                A[high - 1][low - 1] = A[high - 1][low - 1] - 1/value
                A[low - 1][high - 1] = A[low - 1][high - 1] - 1/value

        # elif component.comp_type == 'C':
            # Capacitance is an open circuit for Static Analysis

        elif component.comp_type == 'L':
            # closed circuit  in Static Analysis: 0 resistance and 0 voltage
            # affects the B and C matrices of A
            if high != 0:
                A[high - 1][g2Index] = A[high - 1][g2Index] + 1
                A[g2Index][high - 1] = A[g2Index][high - 1] + 1
            if low != 0:
                A[low - 1][g2Index] = A[low - 1][g2Index] - 1
                A[g2Index][low - 1] = A[g2Index][low - 1] - 1

            # affects b-matrix
            b[g2Index] = 0

            # increase G2 index
            g2Index = g2Index + 1

        elif component.comp_type == 'V':
            # affects the B and C matrices of A
            if high != 0:
                A[high - 1][g2Index] = A[high - 1][g2Index] + 1
                A[g2Index][high - 1] = A[g2Index][high - 1] + 1
            if low != 0:
                A[low - 1][g2Index] = A[low - 1][g2Index] - 1
                A[g2Index][low - 1] = A[g2Index][low - 1] - 1

            # affects b-matrix
            b[g2Index] = value

            # increase G2 counter
            g2Index = g2Index + 1

        elif component.comp_type == 'I':
            # affects b-matrix
            if high != 0:
                b[high - 1] = b[high - 1] - value
            if low != 0:
                b[low - 1] = b[low - 1] + value

    return A, b

In [3]:
def solveSystem(A, b):
    x = np.linalg.solve(A, b)
    return x

In [2]:
# Initialize component counters
vs_c = 0
cs_c = 0
r_c = 0
c_c = 0
i_c = 0

# Parse File
fileName = "Tests/circuit1.sp"
print("Parsing file...\n")
n1 = network.Network()
components = n1.parse_file(fileName)

# Map nodes
print("Mapping nodes...\n")
components = n1.map_nodes(components)

Parsing file...

Mapping nodes...



In [13]:
print("\nNodes are mapped as following:")
for key, val in n1.nodes.items():
    print("\"" + key + "\" :", val)
    


Nodes are mapped as following:
"0" : 0
"5" : 1
"3" : 2
"2" : 3
"7" : 4
"6" : 5
"1" : 6
"12" : 7
"4" : 8

Circuit Components:


TypeError: 'Component' object is not callable

In [21]:


# Print Information
print("Circuit Info")
print("Component count: ", len(components))
print("Voltage count: ", voltageCount)
print("Current count: ", currentCount)
print("Resistance count: ", resistorCount)
print("Capacitance count: ", capacitorCount)
print("Inductance count: ", inductorCount)
print("Node count: ", hashtable.nodeCount)

print("\nNodes are mapped as following:")
for key, val in hashtable.nodes.items():
    print("\"" + key + "\" :", val)

print("\nCircuit Components:")
for i in range(0, len(components)):
    print(components[i])

# Calculate and solve system
print("\nCalculating MNA Matrices...\n")
A, b = calculateMatrices(components, hashtable.nodeCount)
print("A:\n", A)
print("b:\n", b)

print("\nSolving System...\n")
x = solveSystem(A, b)
print("x:\n", x)

Parsing file...

Mapping nodes...

Circuit Info
Component count:  16
Voltage count:  3
Current count:  2
Resistance count:  8
Capacitance count:  2
Inductance count:  1
Node count:  9

Nodes are mapped as following:
"0" : 0
"5" : 1
"3" : 2
"2" : 3
"7" : 4
"6" : 5
"1" : 6
"12" : 7
"4" : 8

Circuit Components:
V 1 0 2.0
V 2 3 0.2
V 4 5 2.0
R 6 1 1.5
R 6 7 1.0
R 1 3 50.0
R 1 5 0.1
R 3 5 1.5
R 2 8 0.2
R 4 0 1000.0
R 8 0 10.0
I 8 4 0.001
I 0 5 0.001
C 4 0 0.1
C 3 0 0.2
L 7 3 0.1

Calculating MNA Matrices...

Group 2 count: 4
Matrix size: 12 

A:
 [[ 10.687   0.     -0.02    0.    -10.     -0.667   0.      0.      1.
    0.      0.      0.   ]
 [  0.      5.      0.      0.      0.      0.      0.     -5.      0.
    1.      0.      0.   ]
 [ -0.02    0.      0.687   0.     -0.667   0.      0.      0.      0.
   -1.      0.     -1.   ]
 [  0.      0.      0.      0.001   0.      0.      0.      0.      0.
    0.      1.      0.   ]
 [-10.      0.     -0.667   0.     10.667   0.      0.      